## Motif Prediction - classically and quantumly

In [1]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import pandas as pd

from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import RealAmplitudes

from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.utils import algorithm_globals

from qiskit.primitives import StatevectorSampler as Sampler

import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import time

In [2]:
neuron_cube_df = pd.read_parquet('neuron_cube_df.parquet')
print(neuron_cube_df.shape)

(5900000, 5)


In [6]:
#neuron_cube_df.info()

In [5]:
neuron_cube_df.describe()

,zDFF,Neuron,Frequency,Window,Stimulus
count,5.900000e+06,5.900000e+06,5.900000e+06,5.900000e+06,5.900000e+06
mean,9.668301e+00,1.180331e+02,5.377561e+00,9.500000e+00,3.083546e-01
std,1.670201e+01,6.906387e+01,1.136437e+01,5.766282e+00,4.618139e-01
min,-1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.405625e-01,5.875000e+01,0.000000e+00,4.750000e+00,0.000000e+00
50%,6.512516e+00,1.175000e+02,0.000000e+00,9.500000e+00,0.000000e+00
75%,1.526041e+01,1.762500e+02,4.243000e+00,1.425000e+01,1.000000e+00
max,6.725551e+02,2.620000e+02,4.800000e+01,1.900000e+01,1.000000e+00


In [10]:
features = neuron_cube_df[['zDFF', 'Neuron', 'Frequency', 'Window']]
labels = neuron_cube_df['Stimulus'].to_numpy()

features = MinMaxScaler().fit_transform(features)

print(type(features), features.shape)
print(type(labels), labels.shape)

<class 'numpy.ndarray'> (5900000, 4)
<class 'numpy.ndarray'> (5900000,)


In [11]:
algorithm_globals.random_seed = 123

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.8, random_state=algorithm_globals.random_seed
)

In [ ]:
svc = SVC()
_ = svc.fit(train_features, train_labels)  # suppress printing the return value

In [ ]:
train_score_c4 = svc.score(train_features, train_labels)
test_score_c4 = svc.score(test_features, test_labels)

print(f"Classical SVC on the training dataset: {train_score_c4:.2f}")
print(f"Classical SVC on the test dataset:     {test_score_c4:.2f}")

### QVC Model

In [ ]:
num_features = features.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)

In [ ]:
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

In [ ]:
optimizer = COBYLA(maxiter=100)

In [ ]:
sampler = Sampler()

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
vqc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    callback=callback_graph,
)

# clear objective value history
objective_func_vals = []

start = time.time()
vqc.fit(train_features, train_labels)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
train_score_q4 = vqc.score(train_features, train_labels)
test_score_q4 = vqc.score(test_features, test_labels)

print(f"Quantum VQC on the training dataset: {train_score_q4:.2f}")
print(f"Quantum VQC on the test dataset:     {test_score_q4:.2f}")

### PCA Feature Reduction?